In [32]:
import genjax
import jax

key = jax.random.PRNGKey(314159)

console = genjax.pretty(show_locals=True, suppress=[])

In [33]:
@genjax.gen
def model(v):
    x = genjax.Normal(v, 1.0) @ "x"
    return x**2

In [34]:
model

BuiltinGenerativeFunction
└── source
    └── PytreeClosure
        ├── callable
        │   └── <function model>
        └── environment
            └── list

In [35]:
prog = genjax.adev.adev(model)

In [36]:
key, v = prog.simulate(key, (1.0,))
v

Array(0.7952519, dtype=float32)

In [37]:
key, out_tangents = prog.grad_estimate(key, (1.0,), (1.0,))
out_tangents

[Array(-2.4382226, dtype=float32)]